### Import reviews

In [1]:
%load_ext autoreload
%autoreload 2

import progressbar

In [2]:
import pickle

DATA_FILE1 = 'data/data.pkl'
DATA_FILE2 = 'data/labels.pkl'
DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'

with open(DATA_FILE1, 'rb') as f:
    reviews = pickle.load(f)
    
with open(DATA_FILE2, 'rb') as f:
    labels = pickle.load(f)
   
entity_names = ['X', 'Y', 'Z']

In [4]:
from babble.babbler import BabbleStream

babbler = BabbleStream(reviews,labels,balanced=True, shuffled=True, seed=456)

Grammar construction complete.


In [ ]:
from babble.utils import display_candidate

candidate = babbler.next()
print(candidate)
print(candidate.text)
print(candidate.entity_positions)

In [5]:
import re

def get_words_before(quantity,sentence,entity):
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = sentence.split()
    if entity in words:
        index = words.index(entity)
        before = index - min(index, quantity)
        return ' '.join(map(str, words[before:index])) 

In [6]:
import re

def get_words_after(quantity,sentence,entity):
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = sentence.split()
    if entity in words:
        index = words.index(entity) +1
        after = index + min(index, quantity)
        return ' '.join(map(str, words[index:after]))

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

pos_adjectives_list=[]
neu_adjectives_list=[]
neg_adjectives_list=[]

def check_sentiment_adjective(adjective):
    if (sid.polarity_scores(adjective)['compound']) >= 0.1:
        pos_adjectives_list.append(adjective)
        return 'positive'
    elif (sid.polarity_scores(adjective)['compound']) <= -0.1:
        neg_adjectives_list.append(adjective)
        return 'negative'
    else:
        neu_adjectives_list.append(adjective)
        return 'neutral' 

In [ ]:
spacy_nlp = spacy.load('en_core_web_sm')
doc = spacy_nlp("please do not eat at this place, i'm scared of this food. the worst part is the service and the trashy, anti-family, anti-american atmosphere. please stay away")
for token in doc:
    if token.pos_ == 'ADJ':
        print(token.text)
        print(check_sentiment_adjective(token.text))

In [ ]:
check_sentiment_adjective('normal')

In [8]:
def check_adjectives_before_pos(sentence,entity):
    words = get_words_before(2,sentence,entity)
    if words == None or len(words.split(" ")) == 0:
        return None
    else:
        spacy_nlp = spacy.load('en_core_web_sm')
        doc = spacy_nlp(words)
        for token in doc:
            if token.pos_ == 'ADJ':
                return token.text
                break

In [9]:
def check_adjectives_after_verb(sentence,entity):
    words = get_words_after(3,sentence,entity)
    if words == None or len(words.split(" ")) < 3:
        return None
    else:
        spacy_nlp = spacy.load('en_core_web_sm')
        doc = spacy_nlp(words)
        if doc[0].pos_ == 'VERB' and (doc[1].pos_ == 'ADJ' or doc[2].pos_ == 'ADJ'):
            return doc[1].text

In [ ]:
def create_condition()

In [ ]:
print(get_words_before(2,candidate.text,"chicken"))

In [ ]:
print(check_adjectives_before_pos(candidate.text,"customer"))

In [10]:
import spacy

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en_core_web_sm')

spaCy Version: 2.1.8


In [11]:
from babble import Explanation

explanations = []
index = 1


for candidate,label in progressbar.progressbar(zip(reviews[0],labels[0])):
#    print(candidate.text)
    for entity,name in zip(candidate.entities,entity_names):
#        print(entity.entity)
        adjective = check_adjectives_after_verb(candidate.text,entity.entity)
        if adjective != None:
            sentiment_value = check_sentiment_adjective(adjective)
            condition='A ' + sentiment_value + ' word is within 3 words to the right of word "' + entity.entity + '"'
        else:
            adjective = check_adjectives_before_pos(candidate.text,entity.entity)
            if adjective != None:
                sentiment_value = check_sentiment_adjective(adjective)
                condition='A ' + sentiment_value + ' word is within 2 words to the left of word "' + entity.entity + '"'
        if adjective != None and sentiment_value != 'neutral':
            explanation = Explanation(
            name='LF_' + str(index),
            label=label,
            condition=condition,
            candidate=candidate,
            )
            explanations.append(explanation)
            index = index + 1
#            print(str(explanation).upper())

| |                          #                      | 300 Elapsed Time: 0:04:02


In [ ]:
for explanation in explanations:
    print(explanation.candidate)

In [ ]:
print(explanations)

In [ ]:
babbler.view_parse(parses[1])

In [ ]:
print(parses[10].semantics)

In [ ]:
parses, filtered = babbler.apply(explanations)

In [ ]:
babbler.commit()

In [ ]:
from metal.analysis import lf_summary

Ls = [babbler.get_label_matrix(split) for split in [0,1,2]]
lf_names = [lf.__name__ for lf in babbler.get_lfs()]
lf_summary(Ls[1], labels[1], lf_names=lf_names)

In [ ]:
print(Ls)

In [12]:
pos_adjectives_list = list(dict.fromkeys(pos_adjectives_list))
neu_adjectives_list = list(dict.fromkeys(neu_adjectives_list))
neg_adjectives_list = list(dict.fromkeys(neg_adjectives_list))

In [13]:
with open(DATA_FILE3, 'wb') as f:
    pickle.dump(pos_adjectives_list,f)
    
with open(DATA_FILE4, 'wb') as f:
    pickle.dump(neu_adjectives_list,f)

with open(DATA_FILE5, 'wb') as f:
    pickle.dump(neg_adjectives_list,f)

In [14]:
from babble.utils import ExplanationIO

FILE = "data/my_explanations.tsv"
exp_io = ExplanationIO()
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)

Wrote 206 explanations to data/my_explanations.tsv
Read 206 explanations from data/my_explanations.tsv
